In [38]:
import sqlite3
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale

In [40]:
cnx = sqlite3.connect('./soccer/database.sqlite')
cursor = cnx.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
#list of all the tables in the schema 
print(cursor.fetchall())

[('sqlite_sequence',), ('Player_Attributes',), ('Player',), ('Match',), ('League',), ('Country',), ('Team',), ('Team_Attributes',)]


In [41]:
df_league = pd.read_sql_query("select * from League;", conn)
df_league.head()
df_player_attributes = pd.read_sql_query("select * from Player_Attributes;", conn)
df_player = pd.read_sql_query("select * from Player;", conn)
df_match = pd.read_sql_query("select * from Match;", conn)
df_country = pd.read_sql_query("select * from Country;", conn)
df_team_attributes = pd.read_sql_query("select * from Team_Attributes;", conn)
df_team = pd.read_sql_query("select * from Team;", conn)

In [42]:
df_player_attributes.head(1)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0


In [43]:
df_player

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
5,6,27316,Aaron Hunt,158138,1986-09-04 00:00:00,182.88,161
6,7,564793,Aaron Kuhl,221280,1996-01-30 00:00:00,172.72,146
7,8,30895,Aaron Lennon,152747,1987-04-16 00:00:00,165.10,139
8,9,528212,Aaron Lennox,206592,1993-02-19 00:00:00,190.50,181
9,10,101042,Aaron Meijers,188621,1987-10-28 00:00:00,175.26,170


In [44]:
df_country.head(1)

,id,name
0,1,Belgium


In [45]:
df_team_attributes.head(1)

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover


In [46]:
df_team.head(1)

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN


In [47]:
df_match.head(1)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.0,1.65,3.4,4.5,1.78,3.25,4.0,1.73,3.4,4.2


In [49]:
df_match.isna().sum()
df_match = df_match[['country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_player_1',
       'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5','home_player_6', 'home_player_7', 'home_player_8', 'home_player_9',
       'home_player_10', 'home_player_11', 'away_player_1', 'away_player_2','away_player_3', 'away_player_4', 'away_player_5', 'away_player_6',
       'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10','away_player_11']]
df_match.isna().sum()

country_id             0
league_id              0
season                 0
stage                  0
date                   0
match_api_id           0
home_team_api_id       0
away_team_api_id       0
home_player_1       1224
home_player_2       1315
home_player_3       1281
home_player_4       1323
home_player_5       1316
home_player_6       1325
home_player_7       1227
home_player_8       1309
home_player_9       1273
home_player_10      1436
home_player_11      1555
away_player_1       1234
away_player_2       1278
away_player_3       1293
away_player_4       1321
away_player_5       1335
away_player_6       1313
away_player_7       1235
away_player_8       1341
away_player_9       1328
away_player_10      1441
away_player_11      1554
dtype: int64

In [50]:
df_team_attributes = df_team_attributes[['id', 'team_api_id', 'date']]
df_team_attributes.isna().sum()

id             0
team_api_id    0
date           0
dtype: int64

In [51]:
df_team.isna().sum()
df_team = df_team[['id', 'team_api_id', 'team_long_name']]
df_team.isna().sum()

id                0
team_api_id       0
team_long_name    0
dtype: int64

In [52]:
df_player.isna().sum()
df_player = df_player[['player_api_id', 'player_name' ]]

In [53]:
df_player_attributes = df_player_attributes[[ 'player_api_id', 'date', 'overall_rating','potential']]
df_player_attributes.isna().sum()

player_api_id       0
date                0
overall_rating    836
potential         836
dtype: int64

In [54]:
df_player_attributes = df_player_attributes.dropna()
df_player_attributes.isna().sum()

player_api_id     0
date              0
overall_rating    0
potential         0
dtype: int64

In [55]:
df_player.head(1)

,player_api_id,player_name
0,505942,Aaron Appindangoye


In [56]:
df_player_attributes.head(1) 

,player_api_id,date,overall_rating,potential
0,505942,2016-02-18 00:00:00,67.0,71.0


In [57]:
df_comb_player =  pd.merge(df_player, df_player_attributes, on="player_api_id")
df_comb_player.head(1)

,player_api_id,player_name,date,overall_rating,potential
0,505942,Aaron Appindangoye,2016-02-18 00:00:00,67.0,71.0


In [58]:
#converting Object to DateTime so we can run the Date Queries using DateTime
df_comb_player['date'] = pd.to_datetime(df_comb_player['date'])

Steps for finding the Top 10 Player
1) Filtering the players by year 
2) Taking the last updated intance of the player attributes of that year
3) Sorting the Players by overall_rating column, if overall_rating for any two player is same we will consider the potential column.
4) Returning the top 10 values from the sorted DataFrame.

In [60]:
def top_N_players(df,year,n=10):
    """Function which return N Number of Top Player at end of Year"""
    df_top = df[df['date'].dt.year == year]
    df_top = df_top.sort_values('date').groupby('player_api_id').last() #Taking only last Instance
    df_top = df_top.sort_values(['overall_rating','potential']).tail(n)
    df_top = df_top.sort_values(['overall_rating','potential'],ascending=False)
    return df_top

In [61]:
df = top_N_players(df_comb_player,2016)[['player_name']]
df.reset_index(level=0, inplace=True)
df.index = range(1,len(df.index)+1)
df.reset_index(level=0, inplace=True)
df.rename( columns={"index":"rank1"},inplace=True)
df_rank =df[:5]
df

,rank1,player_api_id,player_name
0,1,19533,Neymar
1,2,27299,Manuel Neuer
2,3,30834,Arjen Robben
3,4,107417,Eden Hazard
4,5,36378,Mesut Oezil
5,6,93447,Robert Lewandowski
6,7,37412,Sergio Aguero
7,8,164684,James Rodriguez
8,9,182917,David De Gea
9,10,95078,Toni Kroos


In [62]:
df = top_N_players(df_comb_player,2015)[['player_name']]
df.reset_index(level=0, inplace=True)
df.index = range(1,len(df.index)+1)
df.reset_index(level=0, inplace=True)
df.rename( columns={"index":"rank2"},inplace=True)
df_rank = pd.merge(df_rank,df[:5],on=["player_api_id","player_name"],how='outer')
df

,rank2,player_api_id,player_name
0,1,30981,Lionel Messi
1,2,30893,Cristiano Ronaldo
2,3,27299,Manuel Neuer
3,4,30834,Arjen Robben
4,5,40636,Luis Suarez
5,6,19533,Neymar
6,7,107417,Eden Hazard
7,8,35724,Zlatan Ibrahimovic
8,9,37459,David Silva
9,10,80562,Thiago Silva


In [63]:
df = top_N_players(df_comb_player,2014)[['player_name']]
df.reset_index(level=0, inplace=True)
df.index = range(1,len(df.index)+1)
df.reset_index(level=0, inplace=True)
df.rename( columns={"index":"rank3"},inplace=True)
df_rank = pd.merge(df_rank,df[:5],on=["player_api_id","player_name"],how='outer')
df

,rank3,player_api_id,player_name
0,1,30981,Lionel Messi
1,2,30893,Cristiano Ronaldo
2,3,27299,Manuel Neuer
3,4,30834,Arjen Robben
4,5,35724,Zlatan Ibrahimovic
5,6,40636,Luis Suarez
6,7,30955,Andres Iniesta
7,8,107417,Eden Hazard
8,9,30872,Bastian Schweinsteiger
9,10,30924,Franck Ribery


In [65]:
#Making a Line Plot of top 15 Players
df_rank.index = df_rank.player_name
#Taking only the rank in consideration
df_rank = df_rank[[ 'rank1', 'rank2','rank3']]

In [66]:
df_rank = df_rank.replace(np.NaN,6) # Replacing the NaNs with 11 to better represent the graph
df_rank['sum_rank'] = df_rank[[ 'rank1', 'rank2','rank3']].sum(axis=1)

In [67]:
df_rank =  df_rank.sort_values('sum_rank').head(10) #sorting and taking only top 15 values 
thickness = df_rank.sum_rank
df_rank  = df_rank.drop('sum_rank',axis=1)
df1_transposed = df_rank.T

In [68]:
df1_transposed

player_name,Manuel Neuer,Lionel Messi,Cristiano Ronaldo,Arjen Robben,Neymar,Eden Hazard,Mesut Oezil,Luis Suarez,Zlatan Ibrahimovic
rank1,2.0,6.0,6.0,3.0,1.0,4.0,5.0,6.0,6.0
rank2,3.0,1.0,2.0,4.0,6.0,6.0,6.0,5.0,6.0
rank3,3.0,1.0,2.0,4.0,6.0,6.0,6.0,6.0,5.0


In [74]:
df1_transposed.plot(kind='line',figsize=(8,12), marker='o')
plt.gca().invert_yaxis() #inverting y axis
plt.gca().invert_xaxis() #inverting y axis
plt.yticks(range(1,6))
ind = np.arange(4) 
plt.xticks(ind, ("2016","2015","2014"))
plt.show()

'df1_transposed.plot(kind=\'line\',figsize=(8,12), marker=\'o\')\nplt.gca().invert_yaxis() #inverting y axis\nplt.gca().invert_xaxis() #inverting y axis\nplt.yticks(range(1,6))\nind = np.arange(4) \nplt.xticks(ind, ("2016","2015","2014"))\nplt.show()'